In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('example_data.xlsx')
amp=data.amplitude

In [3]:
data.duration=pd.to_timedelta(data.duration.astype(str))

In [4]:
class Sign:
    def __init__(self,data,pn=True):        
        self.__pn = pn        
        
        #df'in sütunları alınıyor        
        self.duration  = data['duration'].dt.seconds
        self.amplitude = data['amplitude']
        self.date      = data['date']       
        
        #kullanılacak fonksiyonlar
        self._median     = None
        self._percentile = None
        self._is_provide = None
        
        #fonksiyona göre atanacak koşul değerleri
        self.duration_val    = None
        self.amplitude_val   = None
        self.duration_bools  = None
        self.amplitude_bools = None
        self.bools = None
        
        #pozitif-negatif durumuna göre atanacak fonksiyonlar
        if self.__pn==False:
            self._median     = self._direct_median
            self._percentile = self._direct_percentile
            self._is_provide = self._direct_is_provide
        elif self.__pn == True:            
            self._median     = self._pn_median
            self._percentile = self._pn_percentile 
            self._is_provide = self._pn_is_provide
            
    def medyan(self):
        self.duration_val  = self._median(self.duration)
        self.amplitude_val = self._median(self.amplitude)
        self.assign_bools()
        return self.sign_data(self.amplitude,self.bools)
        
        
    def percentile(self,percen_val):
        self.duration_val=self._percentile(self.duration,percen_val)
        self.amplitude_val=self._percentile(self.amplitude,percen_val)
        self.assign_bools()
        return self.sign_data(self.amplitude,self.bools)
        
    def assign_bools(self):
        self.duration_bools= self._get_bools(data=self.duration,val=self.duration_val)       
        self.amplitude_bools= self._get_bools(self.amplitude,self.amplitude_val)
        self.bools = self.duration_bools & self.amplitude_bools
        
    def _get_bools(self,data,val) -> pd.Series:
        bools=data.apply(lambda x : self._is_provide(x,val))
        bools[0]=False
        return bools
    
    def _direct_is_provide(self,x,val):
        return abs(x)<=val
    
    def _pn_is_provide(self,x,val):
        if x <0:
            return abs(x)<=val['negatif']
        if x >0:
            return abs(x)<=val['pozitif']
        
    def _pn_median(self,data) -> dict:
        """Pozitif ve negatif'i ayrı hesaplar
        """
        return {'pozitif': data[data>0].median(),'negatif': data[data<0].median()}
        
        
    def _direct_median(self,data):
        return data.median()
    
    def _direct_percentile(self,data,percent_val):
        return data.quantile(percent_val)
        
        
    def _pn_percentile(self,data, percent_val) -> dict:
        """Pozitif ve negatif'i ayrı hesaplar
        """
        return {'pozitif':
            data[data>0].quantile(percent_val),\
                'negatif':data[data<0].quantile(percent_val)}
    
    def sign_data(self,data : pd.Series, bools : pd.Series) -> pd.Series:
        a=data.mask(bools).fillna(method='ffill')
        return np.sign(a).diff().ne(0).cumsum()
                    
        

In [6]:
x=Sign(data,pn=True)

In [8]:
x.duration_val

{'pozitif': 4.5, 'negatif': nan}

In [9]:
x.amplitude_val

{'pozitif': 5.5, 'negatif': -4.5}

In [10]:
data['sign'] = x.medyan()

In [11]:
data

,date,duration,amplitude,sign
0,2018-09-06 00:00:17,00:00:13,-2,1
1,2018-09-06 00:00:39,00:00:22,3,2
2,2018-09-06 00:00:48,00:00:09,-1,3
3,2018-09-06 00:00:49,00:00:01,4,3
4,2018-09-06 00:00:50,00:00:01,-7,3
5,2018-09-06 00:00:51,00:00:01,7,4
6,2018-09-06 00:00:53,00:00:02,-9,5
7,2018-09-06 00:01:01,00:00:08,7,6
8,2018-09-06 00:01:04,00:00:03,-11,7
9,2018-09-06 00:01:05,00:00:01,5,7
